<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_6_Mixing_2D_truss_elements_with_spring_elements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# ============================================================
# Mixing 2D truss elements with spring elements (Problem 6)
# MATLAB -> Python (Google Colab)  |  copy & run in 1 cell
# ============================================================

def solution(GDof, prescribedDof, K, F, u_prescribed=None):
    if u_prescribed is None:
        u_prescribed = np.zeros(len(prescribedDof), dtype=float)

    prescribed = np.array([d-1 for d in prescribedDof], dtype=int)  # 0-based
    all_dofs = np.arange(GDof, dtype=int)
    free = np.setdiff1d(all_dofs, prescribed)

    U = np.zeros(GDof, dtype=float)
    U[prescribed] = u_prescribed

    K_ff = K[np.ix_(free, free)]
    K_fp = K[np.ix_(free, prescribed)]
    F_f = F[free]

    rhs = F_f - K_fp @ U[prescribed]
    U[free] = np.linalg.solve(K_ff, rhs)

    return U


def outputDisplacementsReactions(U, K, GDof, prescribedDof, F):
    R = K @ U - F

    print("=== DISPLACEMENTS (U) ===")
    for i in range(GDof):
        print(f"DOF {i+1}: U = {U[i]:.6e}")

    print("\n=== REACTIONS AT PRESCRIBED DOFs ===")
    for d in prescribedDof:
        print(f"DOF {d}: R = {R[d-1]:.6e}")


# -----------------------------
# Input (same as MATLAB)
# -----------------------------
E = 210000.0
A = 500.0
EA = E * A

# nodeCoordinates = [0 0; -5000*cos(pi/4) 5000*sin(pi/4); -10000 0];
nodeCoordinates = np.array([
    [0.0, 0.0],
    [-5000.0*np.cos(np.pi/4.0),  5000.0*np.sin(np.pi/4.0)],
    [-10000.0, 0.0]
], dtype=float)

# elementNodes = [1 2; 1 3; 1 4];  % spring added => node 4 exists
elementNodes = np.array([
    [1, 2],
    [1, 3],
    [1, 4]
], dtype=int)

numberElements = elementNodes.shape[0]
numberNodes = nodeCoordinates.shape[0] + 1   # +1 for spring node (node 4)

xx = nodeCoordinates[:, 0]
yy = nodeCoordinates[:, 1]

# DOFs
GDof = 2 * numberNodes
U = np.zeros(GDof, dtype=float)
F = np.zeros(GDof, dtype=float)
K = np.zeros((GDof, GDof), dtype=float)

# applied load at node 1, y dof (MATLAB: force(2)=-25000)
F[1] = -25000.0

# -----------------------------
# Assemble truss elements (only up to numberElements-1)
# MATLAB: for e=1:numberElements-1
# -----------------------------
for e in range(numberElements - 1):
    n1, n2 = elementNodes[e, 0], elementNodes[e, 1]  # 1-based
    edof = np.array([n1*2-1, n1*2, n2*2-1, n2*2], dtype=int) - 1  # 0-based

    xa = xx[n2-1] - xx[n1-1]
    ya = yy[n2-1] - yy[n1-1]
    L = np.sqrt(xa*xa + ya*ya)

    C = xa / L
    S = ya / L

    k_local = (EA / L) * np.array([
        [ C*C,  C*S, -C*C, -C*S],
        [ C*S,  S*S, -C*S, -S*S],
        [-C*C, -C*S,  C*C,  C*S],
        [-C*S, -S*S,  C*S,  S*S],
    ], dtype=float)

    K[np.ix_(edof, edof)] += k_local

# -----------------------------
# Add spring stiffness in global DOF
# MATLAB: stiffness([2 7],[2 7]) += 2000*[1 -1; -1 1]
# => DOF2 = uy(node1), DOF7 = uy(node4)
# -----------------------------
k3 = 2000.0
idx = np.array([2, 7], dtype=int) - 1  # -> 0-based [1,6]
K[np.ix_(idx, idx)] += k3 * np.array([[1.0, -1.0],
                                      [-1.0, 1.0]], dtype=float)

# -----------------------------
# Boundary conditions and solution
# MATLAB: prescribedDof=[3:8]
# -----------------------------
prescribedDof = list(range(3, 9))

U = solution(GDof, prescribedDof, K, F)

# Output displacements/reactions
outputDisplacementsReactions(U, K, GDof, prescribedDof, F)

# -----------------------------
# Stresses in truss elements (only truss ones: 1..numberElements-1)
# MATLAB loop in stresses section: for e=1:numberElements-1
# -----------------------------
sigma = np.zeros(numberElements - 1, dtype=float)
for e in range(numberElements - 1):
    n1, n2 = elementNodes[e, 0], elementNodes[e, 1]
    edof = np.array([n1*2-1, n1*2, n2*2-1, n2*2], dtype=int) - 1

    xa = xx[n2-1] - xx[n1-1]
    ya = yy[n2-1] - yy[n1-1]
    L = np.sqrt(xa*xa + ya*ya)

    C = xa / L
    S = ya / L

    sigma[e] = (E / L) * np.array([-C, -S, C, S], dtype=float) @ U[edof]

print("\nstresses")
print(sigma)


=== DISPLACEMENTS (U) ===
DOF 1: U = -1.724138e+00
DOF 2: U = -3.448276e+00
DOF 3: U = 0.000000e+00
DOF 4: U = 0.000000e+00
DOF 5: U = 0.000000e+00
DOF 6: U = 0.000000e+00
DOF 7: U = 0.000000e+00
DOF 8: U = 0.000000e+00

=== REACTIONS AT PRESCRIBED DOFs ===
DOF 3: R = -1.810345e+04
DOF 4: R = 1.810345e+04
DOF 5: R = 1.810345e+04
DOF 6: R = 0.000000e+00
DOF 7: R = 6.896552e+03
DOF 8: R = 0.000000e+00

stresses
[ 51.20428415 -36.20689655]
